# Seq2Seq Prefix-Tuning for C#

This is the main notebook for prefix-tuning *codet5p-220m-bimodal seq2seq* on C# programming language dataset.

## Clone the repository

In [1]:
!git clone https://github.com/leiluk1/CodeSearcher.git && cd CodeSearcher/ && git checkout dev/embeddings

Cloning into 'CodeSearcher'...
remote: Enumerating objects: 251, done.
remote: Counting objects: 100% (251/251), done.
remote: Compressing objects: 100% (163/163), done.
remote: Total 251 (delta 138), reused 188 (delta 83), pack-reused 0
Receiving objects: 100% (251/251), 7.17 MiB | 14.38 MiB/s, done.
Resolving deltas: 100% (138/138), done.
Branch 'dev/embeddings' set up to track remote branch 'dev/embeddings' from 'origin'.
Switched to a new branch 'dev/embeddings'


## Set up the required dependencies

In [ ]:
!pip install dataprep gdown py7zr transformers peft evaluate rouge_score fire loguru --quiet

In [3]:
# import wandb
# wandb.login(key='')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
!mkdir -p /kaggle/output/CodeSearcher/output
!mkdir -p CodeSearcher/data/raw
!gdown 1tZfsYQgWmc2gG340ru5VbrZ5aLIZ41_6
!unzip -q -d /kaggle/working/CodeSearcher/data/raw ./XLCoST_data.zip

Downloading...
From (uriginal): https://drive.google.com/uc?id=1tZfsYQgWmc2gG340ru5VbrZ5aLIZ41_6
From (redirected): https://drive.google.com/uc?id=1tZfsYQgWmc2gG340ru5VbrZ5aLIZ41_6&confirm=t&uuid=43e0cd75-cef7-4754-936a-b646b21fc289
To: /kaggle/working/XLCoST_data.zip
100%|█████████████████████████████████████████| 298M/298M [00:01<00:00, 202MB/s]


## Perform prefix-tuning 

For more details, please check the code in `src/models/train.py` in our repository.

In [5]:
!export DATASETS_VERBOSITY=error
!cd CodeSearcher/ && export PYTHONPATH=. && python src/models/train.py seq2seq prefix \
    --output_dir="output" \
    --epochs=10 \
    --num_virtual_tokens=20 \
    --language="Csharp" \
    --model_max_src_length=256 \
    --model_max_tgt_length=256 \
    --train_batch_size=16 \
    --eval_batch_size=16 \
    --warmup_steps=200 \
    --gradient_accumulation_steps=4 \ 

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
A new version of the following files was downloaded from https://huggingface.co/Salesforce/codet5p-220m-bimodal:
- configuration_codet5p_bimodal.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/Salesforce/codet5p-220m-bimodal:
- modeling_codet5p_bimodal.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
2023-11-18 22:02:30.663 | INFO     | __main__:_setup_peft_model:28 - trainable params: 184,320 || all params: 223,267,074 || trainable%: 0.08255583624480159
20

## Save checkpoint

In [6]:
!zip -r seq2seq_prefix_csharp.zip CodeSearcher/output

  adding: CodeSearcher/output/ (stored 0%)
  adding: CodeSearcher/output/checkpoint-4500/ (stored 0%)
  adding: CodeSearcher/output/checkpoint-4500/rng_state.pth (deflated 28%)
  adding: CodeSearcher/output/checkpoint-4500/merges.txt (deflated 54%)
  adding: CodeSearcher/output/checkpoint-4500/scheduler.pt (deflated 49%)
  adding: CodeSearcher/output/checkpoint-4500/trainer_state.json (deflated 79%)
  adding: CodeSearcher/output/checkpoint-4500/tokenizer_config.json (deflated 81%)
  adding: CodeSearcher/output/checkpoint-4500/training_args.bin (deflated 48%)
  adding: CodeSearcher/output/checkpoint-4500/special_tokens_map.json (deflated 82%)
  adding: CodeSearcher/output/checkpoint-4500/vocab.json (deflated 59%)
  adding: CodeSearcher/output/checkpoint-4500/tokenizer.json (deflated 72%)
  adding: CodeSearcher/output/checkpoint-4500/README.md (deflated 65%)
  adding: CodeSearcher/output/checkpoint-4500/added_tokens.json (deflated 37%)
  adding: CodeSearcher/output/checkpoint-4500/adapte

## Evaluation

In this step, test the model using *Mean Reciprocal Rank (MRR)* as the evaluation metric. 

For more details, please refer to `src/models/evaluation.py` in the repository.

In [10]:
!cd CodeSearcher/ && export PYTHONPATH=. && python src/models/evaluation.py \
    --tuned_ckpt_path="output/best_ckpt/" \
    --num_virtual_tokens=20 \
    --language="Csharp" \
    --model_max_src_length=128 \
    --model_max_tgt_length=128 \
    --batch_size=32

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
A new version of the following files was downloaded from https://huggingface.co/Salesforce/codet5p-220m-bimodal:
- configuration_codet5p_bimodal.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/Salesforce/codet5p-220m-bimodal:
- modeling_codet5p_bimodal.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
2023-11-21 19:32:07.643 | INFO     | src.datasets.XLCoST.make_dataset:_load_search_dataframe:84 - Loading dataframe from data/raw/XLCoST_data/retrieval/nl2code

# Results:

- Test MRR for C#: **0.12993185**;
- Epochs: **10**;
- Trainable params: 184,320 || all params: 223,267,074 || trainable%: 0.08255583624480159.

You can also check the validation plots in [wanb report](https://api.wandb.ai/links/ley-khaertdinova/13lvn64p).